In [1]:
from imports import *
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
%matplotlib inline

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [8]:
def read_NASA_stellar_data():
    d = np.genfromtxt('input_data/NASAexoplanet_Aug222018_stellarFGK.csv', skip_header=53, delimiter=',')
    print d.shape
    ids,_,_,_,ra,_,dec,dist,udist,ldist,_,Teff,uTeff,lTeff,_,Ms,uMs,lMs,_,Rs,uRs,lRs,_,_,Ls,uLs,lLs,_,M_H,uM_H,lM_H,_,Vmag,eVmag,_,Jmag,eJmag,_,Hmag,eHmag,_,Kmag,eKmag,_ = d.T
    _,names = np.genfromtxt('input_data/NASAexoplanet_Aug222018_stellarFGK.csv', skip_header=53, delimiter=',', usecols=(0,1), dtype='|S30').T
    return ids, names, ra, dec, dist, udist, ldist, Teff, uTeff, lTeff, Ms, uMs, lMs, Rs, uRs, lRs, Ls, uLs, lLs, M_H, uM_H, lM_H, Vmag, eVmag, Jmag, eJmag, Hmag, eHmag, Kmag, eKmag

In [9]:
# get data
ids, names, ra, dec, dist, udist, ldist, Teff, uTeff, lTeff, Ms, uMs, lMs, Rs, uRs, lRs, Ls, uLs, lLs, M_H, uM_H, lM_H, Vmag, eVmag, JVmag, eJmag, Hmag, eHmag, Kmag, eKmag = read_NASA_stellar_data()
print 'Nentries =', ids.size

(2696, 44)
Nentries = 2696


In [10]:
def query_gaia(ra_deg, dec_deg, rad_arcsec=5):
    coord = SkyCoord(ra=ra_deg, dec=dec_deg, unit=(u.degree, u.degree), frame='icrs')
    r = Gaia.query_object_async(coordinate=coord, radius=u.Quantity(rad_arcsec, u.arcsecond))
    if len(r) > 0:
        i = np.where(r['dist'].base == np.min(r['dist'].base))[0][0]
    else:
        raise ValueError('No solutions. radius = %.1f arcsec'%rad_arcsec)
    ra, dec, par, epar = r[i]['ra'], r[i]['dec'], r[i]['parallax'], r[i]['parallax_error']
    return ra, dec, par, epar

In [ ]:
# query gaia for each star
#_,inds = np.unique(names, return_index=True)
results = np.zeros((ids.size,5)) + np.nan
results[:,0] = ids
for i in range(ids.size):
    print float(i)/ids.size
    try:
        results[i,1:] = query_gaia(ra[i], dec[i], 10)
    except ValueError:
        pass

0.0


Query finished.
0.000370919881306
Query finished.
0.000741839762611
Query finished.
0.00111275964392
Query finished.
0.00148367952522
Query finished.
0.00185459940653
Query finished.
0.00222551928783
Query finished.
0.00259643916914
Query finished.
0.00296735905045
Query finished.
0.00333827893175
Query finished.
0.00370919881306
Query finished.
0.00408011869436
Query finished.
0.00445103857567
Query finished.
0.00482195845697
Query finished.
0.00519287833828
Query finished.
0.00556379821958
Query finished.
0.00593471810089
Query finished.
0.0063056379822
Query finished.
0.0066765578635
Query finished.
0.00704747774481
Query finished.
0.00741839762611
Query finished.
0.00778931750742
Query finished.
0.00816023738872
Query finished.
0.00853115727003
Query finished.
0.00890207715134
Query finished.
0.00927299703264
Query finished.
0.00964391691395
Query finished.
0.0100148367953
Query finished.
0.0103857566766
Query finished.
0.0107566765579
Query finished.
0.0111275964392
Query finished

In [7]:
# save results
hdr = 'ID, RA [deg], Dec [deg], Parallax [mas], eParallax [mas]'
np.savetxt('input_data/GAIA_query_resultsFGK.csv', results, delimiter=',', fmt='%.15f', header=hdr)

## old stuff

In [8]:
# create query form
_,inds = np.unique(names, return_index=True)
names2, ra2, dec2 = names[inds], ra[inds], dec[inds]
g1, g2 = '', ''
for i in range(names2.size):
    label = '+' if dec2[i] >= 0 else ''
    g1 += '%s\t%.7f%s%.7f (degrees, degrees)\n'%(names2[i], ra2[i], label, dec2[i])
    g1 += '%.7f %.7f\n'%(ra2[i], dec2[i])
    g2 += '%s\n'%names2[i]
    
f = open('input_data/gaia_query.txt', 'w')
f.write(g2)
f.close()

f = open('input_data/gaia_query_crosscheck.txt', 'w')
f.write(g1)
f.close()

In [9]:
print g1

EPIC 211822797	130.4103700+17.6400050 (degrees, degrees)
130.4103700 17.6400050
GJ 1132	153.7157370-47.1567270 (degrees, degrees)
153.7157370 -47.1567270
GJ 1214	258.8288370+4.9639200 (degrees, degrees)
258.8288370 4.9639200
GJ 3470	119.7743490+15.3914920 (degrees, degrees)
119.7743490 15.3914920
GJ 9827	351.7701420-1.2862730 (degrees, degrees)
351.7701420 -1.2862730
HAT-P-54	99.8980030+25.4825400 (degrees, degrees)
99.8980030 25.4825400
HATS-6	88.1467970-19.0316560 (degrees, degrees)
88.1467970 -19.0316560
K2-104	129.6367490+19.7738290 (degrees, degrees)
129.6367490 19.7738290
K2-116	336.1516110-11.5786910 (degrees, degrees)
336.1516110 -11.5786910
K2-117	133.8556820+10.4691310 (degrees, degrees)
133.8556820 10.4691310
K2-120	132.1429900+17.1906850 (degrees, degrees)
132.1429900 17.1906850
K2-122	126.4763110+20.3595700 (degrees, degrees)
126.4763110 20.3595700
K2-123	134.4441830+21.4535330 (degrees, degrees)
134.4441830 21.4535330
K2-124	133.6411900+23.1328910 (degrees, degrees)
133.6